# TSP

In [1]:
# Encode the TSP using qlasskit and parameters

from qlasskit import qlassf, Parameter, Qlist, Qmatrix, Qint


@qlassf
def tsp(
    dst_matrix: Parameter[Qmatrix[Qint[3], 3, 3]], order: Qlist[Qint[2], 3]
) -> Qint[4]:
    dst_sum = Qint4(0)
    for i in range(2):
        oim = order[i]
        oi = order[i + 1]
        dst_sum += dst_matrix[oim][oi]
    return dst_sum


tsp_f = tsp.bind(dst_matrix=[[0, 1, 1], [1, 0, 1], [1, 1, 0]])
tsp_f.expressions
# bqm = tsp_f.to_bqm()

ExpressionNotHandledException: 

In [2]:
# Encode the TSP using qlasskit and parameters

from qlasskit import qlassf, Parameter, Qlist, Qmatrix, Qint


@qlassf
def tsp(dst_matrix: Parameter[Qlist[Qint[3], 9]], order: Qlist[Qint[2], 3]) -> Qint[4]:
    dst_sum = Qint4(0)
    for i in range(2):
        oim = order[i]
        oi = order[i + 1]
        v = oim * 3 + oi
        dst_sum += dst_matrix[v]
    return dst_sum


tsp_f = tsp.bind(dst_matrix=[0, 1, 1, 1, 0, 1, 1, 1, 0])
tsp_f.expressions
# bqm = tsp_f.to_bqm()

ExpressionNotHandledException: 